In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import *
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn import metrics

# Importing the Arxiv Data

We're going to use a bunch of Arxiv physics papers for this task.

In [2]:
# We're already in the directory with the papers, so we can use os.listdir() to get the file names
filename_list = os.listdir()[0:1000]

# There are a few extraneous files we don't want to get caught in the 
# filenamelist, so we add some logic to exclude those
filename_list = [filename for filename in filename_list if len(filename) < 8]

In [3]:
# Check that these file names are correct:
filename_list[0:5]

['0301116', '0304232', '0303017', '0303225', '0302131']

Now we can read in all the files from the `filename_list`.

In [4]:
corpus = []

for i in range(len(filename_list)):
    
    filename = filename_list[i]
    
    # errors='ignore' is added to deal with UnicodeDecodeErrors  
    with open(filename, 'r', errors='ignore') as file:
            file_contents = file.read()
          
    # Add document to corpus
    corpus.append(file_contents)

In [5]:
# Removing LaTeX and other formatting artifacts that will cause issues with NMF and LDA

from nltk.stem.wordnet import WordNetLemmatizer
import re
import gensim.parsing.preprocessing as genpre

lmtzr = WordNetLemmatizer()

def prep_text(text):
     # this removes LaTeX formatting, citations, splits hyphens
    myreg = r'\\[\w]+[\{| ]|\$[^\$]+\$|\(.+\, *\d{2,4}\w*\)|\S*\/\/\S*|[\\.,\/#!$%\^&\*;:{}=_`\'\"~()><\|]|\[.+\]|\d+|\b\w{1,2}\b'
    parsed_data = text.replace('-', ' ')
    parsed_data = re.sub(myreg, '', parsed_data)
    parsed_data = [lmtzr.lemmatize(w) for w in parsed_data.lower().split() if w not in genpre.STOPWORDS]
    return parsed_data

In [6]:
corpus = [prep_text(document) for document in corpus]

`prep_text` didn't remove -everything-, but we will have many fewer artifacts than if we didn't run it at all. We can also scrape off some very common LaTeX phrases by passing them as stopwords when retraining the `TfIdfVectorizer`, and also by setting `max_df` to exclude words that occur in more than 90% of documents.

See this [excellent blog post](https://medium.com/@omar.abdelbadie1/processing-text-for-topic-modeling-c355e907ab23) on why `prep_text` works to remove LaTeX artifacts. All credit goes to author Omar Abdelbadie for this method.

Note that by using `prep_text` we've caused every entry in `corpus` to become a list containing a number of strings, rather than one big string for each entry. This is a problem for when we want to create our feature matrix, as `TfIdfVectorizer` is not compatible with a list of lists. We'll need to use `join` (a string method) to change each entry back to a string instead of a list.

In [7]:
for i in range(len(corpus)):
    corpus[i] = ' '.join(corpus[i])

In [8]:
len(corpus)

997

# Creating a Feature Matrix

Now we have 997 documents to work with. Now we can turn our corpus into a matrix of Term Frequency Inverse Document Frequency (TF-IDF) features using `sklearn`'s `TfidfVectorizer()`.

In [9]:
# Again ignoring any UnicodeDecodeErrors
vectorizer = TfidfVectorizer(decode_error = 'ignore', max_df = 0.9, ngram_range = (1, 2), max_features = 20000)

X = vectorizer.fit_transform(corpus)
X

<997x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 908182 stored elements in Compressed Sparse Row format>

Let's take a look at the vocabulary that was learned by the vectorizer.

In [10]:
# Cast the vocab dict to a list so we can print just a subset of the dict

first20_vocab = {k: vectorizer.vocabulary_[k] for k in list(vectorizer.vocabulary_)[:20]}
first20_vocab

{'latex': 10101,
 'file': 6930,
 'paper': 12957,
 'documentstylerevtex': 4967,
 'documentstylearticle': 4965,
 'beqequation': 1288,
 'eeqequation': 5273,
 'beqaeqnarray': 1286,
 'eeqaeqnarray': 5272,
 'partial': 13043,
 'lraleftrightarrow': 10768,
 'footnote': 7205,
 'thefootnotefootnote': 18056,
 'theequationsectionequation': 18051,
 'defnonumber': 4122,
 'deffootnote': 4044,
 'def': 3970,
 'original': 12836,
 'draft': 5042,
 'flushright': 7079}

And the stopwords:

In [11]:
# Similar to above, use itertools to avoid printing the entire (massive) set to screen
import itertools

for i, val in enumerate(itertools.islice(vectorizer.stop_words_, 20)):
    print(i, val)

0 guaranteed killing
1 eqnarray jdef
2 follows conventional
3 embigl circcal
4 broomstick hand
5 sphere derived
6 eqn nipi
7 member elementary
8 ovl
9 otto stark
10 group prove
11 corresp
12 left coshlambdaright
13 altscen
14 zero leftdel
15 topological case
16 manifold providing
17 constraint possible
18 leftprime
19 ncond operator


A number of LaTeX and nonsense terms, as well as some physics terms, were caught in the filter created by `max_df`. The benefit should outweigh the cost of excluding these particular physics terms. (After all, they must not be very distinctive phrases if they're occurring in 90% of papers.)

The top of the vocabulary showed some additional phrases that occur frequently and are not informative to us. We'll go ahead and remove those low-hanging fruit using by retraining the vectorizer and passing these stop-phrases as a list.

(Normally it would not make sense to slice a dictionary this way, but after having run the vectorizer repeatedly and seeing the order the terms are stored in memory, we can make the call to exclude the 15 phrases that tend to float to the top.)

In [12]:
additional_stopwords = list(vectorizer.vocabulary_)[0:20]

In [13]:
vectorizer = TfidfVectorizer(decode_error='ignore', max_df=0.9, ngram_range=(1, 2), 
                             max_features = 20000, stop_words=additional_stopwords)

X = vectorizer.fit_transform(corpus)
X

<997x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 902123 stored elements in Compressed Sparse Row format>

Now we'll do some topic modeling.

# Topic Modeling

In [14]:
# Initialize NMF
nmf_model = NMF(n_components = 10, solver = 'mu')

# Create variable to make it easy to retrieve topics
idx_to_word = np.array(vectorizer.get_feature_names())

In [15]:
nmf_model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=10, random_state=None, shuffle=False, solver='mu',
  tol=0.0001, verbose=0)

In [16]:
nmf_components = nmf_model.components_

In [17]:
for i, topic in enumerate(nmf_components):
    print("Topic {}: {}".format(i + 1, ", ".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))

Topic 1: operator, symmetry, case, form, function, model, space, term, gauge, phys
Topic 2: eeq, string, cosmological, citation hep, citation, hep, solution, bulk, branes, brane
Topic 3: metric, cosmological, schwarzschild, solution, sitter, entropy, horizon, hole, black hole, black
Topic 4: array, right equation, right right, eqnarray, equation left, left left, right left, left right, right, left
Topic 5: lett citation, jhep, jhep citation, arxivhep citation, arxivhep, phys citation, hep citation, hep, citation hep, citation
Topic 6: string theory, light cone, cone, plane, wave background, state, plane wave, background, wave, string
Topic 7: closed string, energy, sen, rolling tachyon, potential, kink, rolling, inflation, string, tachyon
Topic 8: ghost, gauge, action, delta, phi, nonumber, big, gamma, eqnarray eqnarray, eqnarray
Topic 9: operator, product, star, noncommutative space, quantum, space, noncommutativity, commutative, algebra, noncommutative
Topic 10: vacuum, curve, lref, 

Exciting! We have a few topics here that are composed of LaTeX specifications, but the others are clearly relevant to particular areas of physics.

Let's try out some LDA as well.

In [18]:
lda_model = LatentDirichletAllocation(max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda_model.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [19]:
for i, topic in enumerate(lda_model.components_):
    print("Topic {}: {}".format(i + 1, ", ".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))

Topic 1: equation phys, explicitly equation, eq, inherent, note particular, zi zi, brane model, wm, ortiz, tba
Topic 2: string, constant, citation, matrix model, practical, black hole, sphere, align, gauge, lref
Topic 3: angle decorsize, plainleft, distmm, distmm angle, decorsize, fmfgraph, label distmm, decorsize label, wh wh, wh
Topic 4: branes, state, citation hep, right, left, citation, hep, boundary, brane, string
Topic 5: left, fixed equation, eea, kato, big, difficult, gamma, psi, eqnarray, string
Topic 6: array, citation, right, left, model, operator, phys, gauge, string, eqnarray
Topic 7: sphere, tau, cubic string, bare parameter, fuzzy, physrev, reuter, kac, path path, fuzzy sphere
Topic 8: charge, energy, term, tachyon, branes, action, left, right, wall, brane
Topic 9: ext, form, interaction, gauge, color, mechanic, eqnarray, right, left, phys
Topic 10: solution, citation, gauge, right, brane, hep, left, eqnarray, string, phys


Sadly, the topics generated by LDA are not very interesting or distinct. It looks like NMF is a more appropriate topic modeling method for this dataset.

# Clustering 

Now we're going to do some clustering. In order to get an appropriate matrix, we'll use NMF's `fit_transform` method.

In [20]:
X2 = nmf_model.fit_transform(X)

# First try with 4
kmeans4 = KMeans(n_clusters=4, random_state=6).fit(X2)
kmeans4.labels_

array([2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1,
       3, 0, 0, 1, 2, 3, 1, 2, 3, 1, 1, 1, 3, 1, 1, 2, 3, 2, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       3, 3, 2, 1, 2, 3, 1, 1, 2, 0, 1, 1, 0, 2, 2, 1, 3, 1, 3, 3, 1, 2,
       1, 1, 1, 3, 1, 2, 1, 2, 3, 1, 1, 2, 2, 1, 1, 2, 1, 0, 1, 3, 1, 0,
       1, 1, 3, 1, 1, 1, 1, 3, 0, 3, 1, 2, 1, 3, 1, 3, 1, 1, 1, 1, 3, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 2, 1, 2, 1, 2, 1,
       2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 1, 3, 2, 1, 1, 1, 1, 2, 1,
       2, 1, 1, 3, 1, 1, 1, 1, 3, 2, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 3,
       1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 2, 3, 1, 1, 1, 1, 1, 1,
       3, 0, 1, 3, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 3, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 0, 1, 3, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 1, 0, 1,

In [21]:
# Next, 8 clusters
kmeans8 = KMeans(n_clusters=8, random_state=6).fit(X2)

In [22]:
# And finally, 12
kmeans12 = KMeans(n_clusters=12, random_state=6).fit(X2)

In [23]:
kmeans4.cluster_centers_

array([[0.06046452, 0.0060843 , 0.00682197, 0.00705394, 0.01819956,
        0.02579614, 0.00946373, 0.00929777, 0.0187972 , 0.01835133],
       [0.02521794, 0.0122218 , 0.19401777, 0.00199063, 0.02122031,
        0.01910719, 0.00594836, 0.01326531, 0.00317477, 0.00561036],
       [0.04134829, 0.12306222, 0.01429934, 0.01457846, 0.01819093,
        0.0218589 , 0.06239315, 0.01193312, 0.00334262, 0.02014902],
       [0.04489066, 0.00243255, 0.00605031, 0.09647825, 0.01189382,
        0.01641603, 0.00836684, 0.09700811, 0.02856672, 0.01375213]])

Let's look at some of the metrics to see which number of clusters is most ideal for our dataset.

In [27]:
def get_metrics(km):
    print("Metrics for %s clusters: " % km.get_params('n_clusters'))
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
    print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
    print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
    print("Adjusted Rand-Index: %.3f"
          % metrics.adjusted_rand_score(labels, km.labels_))
    print("Silhouette Coefficient: %0.3f"
          % metrics.silhouette_score(X, km.labels_, sample_size=1000))
    
for model in [kmeans4, kmeans8, kmeans12]:
    get_metrics(model)

Metrics for {'algorithm': 'auto', 'copy_x': True, 'init': 'k-means++', 'max_iter': 300, 'n_clusters': 4, 'n_init': 10, 'n_jobs': None, 'precompute_distances': 'auto', 'random_state': 6, 'tol': 0.0001, 'verbose': 0} clusters: 


NameError: name 'labels' is not defined

In [28]:
if not opts.use_hashing:
    print("Top terms per cluster:")

    if opts.n_components:
        original_space_centroids = svd.inverse_transform(km.cluster_centers_)
        order_centroids = original_space_centroids.argsort()[:, ::-1]
    else:
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vectorizer.get_feature_names()
    for i in range(true_k):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

NameError: name 'opts' is not defined

In [34]:
order_centroids = kmeans4.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(4):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

Cluster 0: aa abc abdus abdus salam abbreviation abcd abdalla ab aai aaa
Cluster 1: aai aa abbreviation abc abdalla aaa abcd abdus salam abdus ab
Cluster 2: aaa abcd aa abc abdus salam abbreviation ab aai abdalla abdus
Cluster 3: abdalla ab aa abdus abc abdus salam abbreviation abcd aai aaa
